In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib
from glob import glob
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.applications import VGG16
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from PyInquirer import prompt, print_json
from keras.models import model_from_json

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# loading the directories 
training_dir = '/visual/Semi_train'
test_dir = '/visual/Semi_test'
#test_dir = '../input/fruits/fruits-360_dataset/fruits-360/test-multiple_fruits/'

In [3]:
# useful for getting number of files
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

In [4]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 11


In [5]:
choices= ['VGG16', 'VGG19', 'ResNet50', 'InceptionV3', 'MobileNet']
print("Please Type one of these options", choices)
model_name = input("Which Model: ");



Please Type one of these options ['VGG16', 'VGG19', 'ResNet50', 'InceptionV3', 'MobileNet']
Which Model: ResNet50


In [6]:
if (model_name == "VGG16"):

    IMAGE_SIZE = [64, 64]  
    vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    model = Model(inputs = vgg.input, outputs = x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
elif(model_name =="VGG19"):

    IMAGE_SIZE = [64, 64]  
    vgg = VGG19(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    model = Model(inputs = vgg.input, outputs = x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

elif(model_name =="ResNet50"):
    IMAGE_SIZE = [64, 64]  
    rnet = ResNet50(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG
    for layer in rnet.layers:
        layer.trainable = False
    x = Flatten()(rnet.output)
    x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    model = Model(inputs = rnet.input, outputs = x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


elif(model_name =="InceptionV3"):
    
    IMAGE_SIZE = [100, 100]  
    Inception = InceptionV3(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (100,100,3) as required by VGG
    for layer in Inception.layers:
        layer.trainable = False
    x = Flatten()(Inception.output)
    x = Dense(num_classes, activation = 'softmax')(x) 
    model = Model(inputs = Inception.input, outputs = x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

elif(model_name =="MobileNet"):
    
    IMAGE_SIZE = [128, 128]  
    mnet = MobileNet(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (128,128,3) as required by MobileNet
    for layer in mnet.layers:
        layer.trainable = False
    x = Flatten()(mnet.output)
    x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.
    model = Model(inputs = mnet.input, outputs = x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 12s 0us/step


In [7]:
print(model_name)
model.summary()


ResNet50
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
a

In [8]:
# Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    preprocessing_function=preprocess_input)


training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode= 'categorical')


Found 40264 images belonging to 11 classes.
Found 11876 images belonging to 11 classes.


In [9]:
# The labels are stored in class_indices in dictionary form. 
# checking the labels
training_generator.class_indices

{'Chiffon': 0,
 'Dress': 1,
 'Jeans': 2,
 'Lace_Dress': 3,
 'Lace_Shirt': 4,
 'Leggings_and_Jeans': 5,
 "Men's_Coats": 6,
 "Men's_Pants": 7,
 "Men's_Shirts": 8,
 "Men's_Summer_Wear": 9,
 "Men's_T_Shirt": 10}

In [10]:
#training_images = 15000
#validation_images = 12709

history = model.fit_generator(training_generator,
                              steps_per_epoch = 201, 
                              epochs = 5,
                              validation_data = test_generator,
                              validation_steps= 59)


Epoch 1/5
201/201 [==============================] - 1522s 8s/step - loss: 1.9612 - acc: 0.4453 - val_loss: 2.0100 - val_acc: 0.3661
Epoch 2/5
201/201 [==============================] - 1410s 7s/step - loss: 1.6353 - acc: 0.4951 - val_loss: 1.9977 - val_acc: 0.3668
Epoch 3/5
201/201 [==============================] - 1400s 7s/step - loss: 1.5705 - acc: 0.5187 - val_loss: 1.8736 - val_acc: 0.3650
Epoch 4/5
201/201 [==============================] - 1400s 7s/step - loss: 1.5434 - acc: 0.5246 - val_loss: 1.8784 - val_acc: 0.3661
Epoch 5/5
201/201 [==============================] - 1390s 7s/step - loss: 1.5175 - acc: 0.5295 - val_loss: 2.0710 - val_acc: 0.3678


In [11]:
temp = image.load_img('/visual/Semi_test/Dress',target_size=(64,64));

In [12]:
temp = image.img_to_array(temp)
temp = np.expand_dims(temp, axis=0)
result = model.predict(temp)


In [13]:
result

array([[1.7496372e-03, 7.5240380e-01, 1.9540951e-06, 1.5645752e-06,
        2.1387929e-01, 1.3659112e-02, 2.0113880e-07, 1.7334158e-03,
        2.8987727e-04, 1.4865283e-02, 1.4159923e-03]], dtype=float32)

In [14]:
training_generator.class_indices

{'Chiffon': 0,
 'Dress': 1,
 'Jeans': 2,
 'Lace_Dress': 3,
 'Lace_Shirt': 4,
 'Leggings_and_Jeans': 5,
 "Men's_Coats": 6,
 "Men's_Pants": 7,
 "Men's_Shirts": 8,
 "Men's_Summer_Wear": 9,
 "Men's_T_Shirt": 10}

In [16]:
#Reference: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
if (model_name == "VGG16"):

    # serialize model to JSON
    model_json = model.to_json()
    with open("Dataset/MyVersion/VGG16.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")
    
elif(model_name =="VGG19"):

    # serialize model to JSON
    model_json = model.to_json()
    with open("Dataset/MyVersion/VGG19.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

elif(model_name =="ResNet50"):
     # serialize model to JSON
    model_json = model.to_json()
    with open("/visual/Result/ResNet50.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

elif(model_name =="InceptionV3"):
    
     # serialize model to JSON
    model_json = model.to_json()
    with open("Dataset/MyVersion/InceptionV3.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

elif(model_name =="MobileNet"):
    
     # serialize model to JSON
    model_json = model.to_json()
    with open("Dataset/MyVersion/MobileNet.json", "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

Saved model to disk
